<a href="https://colab.research.google.com/github/balajisriraj/Twitter-Summary-mini-Project/blob/main/Twitter_Trend_Finder_In_Progress_V0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Flow:

- Connect to Twittter API using the required creds
- Find the Top trending Hashtags
- Then filter the hashtags which are in English language
- Filter Top 3 Hashtags from that
- For these 3 hashtags, fetch 100 Tweets individually
- Combain all the 100 tweets into one big corpus
- Do basic Data Cleaning & Processing for better Tokenization
- Using Text to Text transformer t5 model Summarize the tweets
- Show Summarized text output for each Hashtag

In [5]:
!pip install googletrans

     |████████████████████████████████| 61kB 2.4MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
     |████████████████████████████████| 962kB 8.2MB/s 
     |████████████████████████████████| 61kB 6.5MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
     |████████████████████████████████| 102kB 5.3MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-cp36-none-any.whl size=15736 sha256=5fe14f99277152d4e4eb6cd1f5da8ac7683faa26814c3c23e52dac9becc7e7af
  Stored in directory: /root/.cache/pip/wheels/28/1a/a7/eaf4d7a3417a0c65796c547cff4deb6d79c7d14c2abd29273e
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=27dcba0fc969f83b7a566fc82d40b5f4d278afe90cbba6746f21d880c1ba3a85
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built googletrans contextvars


Import Python Modules

In [53]:
import pandas as pd
import tweepy
from tweepy import OAuthHandler
from tweepy import API
from googletrans import Translator
import datetime
import copy
import string
import re

In [8]:
#Past the keys from your local drive
consumer_key = '3YvJiFS5G13byIL9Kplm2gfOk'
consumer_secret = 'eqmwYNrC7EGOyACIGmEwCErdWotSkzYBUnzJRUPtMVDKZOn1z6'
access_token = '278398045-CLb3QRJeFpREuSnLbbvERWBigJMTf4dsfLXD6LtI'
access_token_secret = 'krACZJgoBisTsK6T1ikYSCZNVnvTlScb0vSV3kVnE42Wi'

In [9]:
# Consumer key authentication(consumer_key,consumer_secret can be collected from our twitter developer profile)
auth = OAuthHandler(consumer_key, consumer_secret)
# Access key authentication(access_token,access_token_secret can be collected from our twitter developer profile)
auth.set_access_token(access_token, access_token_secret)
# Set up the API with the authentication handler
api = API(auth)

In [10]:
WOE_ID = 2295424 # Where on Earth id can be extracted from https://nations24.com/world-wide
lan_find = Translator() # for finding teh language of the hashtags

In [11]:
def get_location_trends(locations, auth,n_hashtags,lang):
  api = API(auth)
  trends = api.trends_place(locations)
  data = trends[0]
  trends_data = data['trends']
  global tred_data
  tred_data = []
  for info in trends_data:
    tred_data.append([info['name'],info['tweet_volume'],lan_find.detect(info['name']).lang ] )
  tred_data = pd.DataFrame(tred_data, columns = list(['Hashtag',
                                                    'Tweet_Volume', 'Language'])).sort_values(by = ['Tweet_Volume'],ascending = False)
  #select english language trends
  tred_data = tred_data[tred_data.Language == lang]
  #select top 3 trends
  tred_data = tred_data.nlargest(n_hashtags,columns=['Tweet_Volume'])
  return tred_data

In [12]:
df_trending = get_location_trends(WOE_ID,auth,3,'en')
df_trending

,Hashtag,Tweet_Volume,Language
37,#PCAs,2098175.0,en
10,President Trump,1499208.0,en
20,#BTSonFallon_D5,330983.0,en


In [13]:
hashtag_list = list(df_trending['Hashtag'])

In [34]:
def extract_tweets_for_htags(no_of_tweets):
  dict_of_df = {} 
  for htag in hashtag_list:
    today = datetime.datetime.now().strftime("%Y-%m-%d")
    tweets = tweepy.Cursor(api.search,q=htag,lang="en",since=today).items(no_of_tweets) #Extracting tweets for the htag
    tweets= [tweet.text for tweet in tweets] # Saving the tweets as list
    
    
    key_name = 'df_htag_'+str(htag)
    dict_of_df[key_name] = copy.deepcopy(tweets)
  return dict_of_df

In [35]:
dict_of_df = extract_tweets_for_htags(3)

In [39]:
hashtag_df_list = dict_of_df.keys() #List contains individual the key names for accessing the data inside the dictionary

In [43]:
temp_list = dict_of_df['df_htag_#PCAs']

In [59]:
def remove_pattern(text, pattern):
    clean_text = []
    for tweet in text:
      r = re.findall(pattern, tweet)
      for i in tweet:
          input_txt = re.sub(i, '', tweet)
          clean_text.append(text)
          break
    return clean_text

In [60]:
remove_pattern(text = temp_list, pattern = "@[\w]*")

[]

In [51]:
def remove_punct(text):
    clean_text = []
    for tweet in text:
      text  = "".join([char for char in tweet if char not in string.punctuation])
      text = re.sub('[0-9]+', '', text)
      clean_text.append(text)
    return clean_text

In [54]:
remove_punct(temp_list)

['RT WorldwideBLINK I vote IceCream by BLACKPINK and selenagomez for TheMusicVideo on this years Peoples Choice Awards PCAs',
 'bpbillboard thebluesies BLACKPINK I vote BLACKPINK\xa0￼\xa0 for TheGroup on this years Peoples Choice Awards PCAs BLACKPINK',
 'RT fetishforsell Since yall are here lets help selena win the award since she had been supporting the pinks since the beginning \n\nRETWE…']

In [55]:
temp_list

["RT @WorldwideBLINK: I vote #IceCream by @BLACKPINK and @selenagomez for #TheMusicVideo on this year's People's Choice Awards #PCAs",
 "@bpbillboard @thebluesies @BLACKPINK I vote #BLACKPINK\xa0￼\xa0 for #TheGroup on this year's People's Choice Awards #PCAs @BLACKPINK",
 "RT @fetishforsell: Since y'all are here let's help selena win the award since she had been supporting the pinks since the beginning \n\nRETWE…"]